# Getting Started with OpenHands SDK

This notebook is an introduction to building agents with the [OpenHands SDK](https://docs.openhands.dev/sdk).

**What you'll learn:**
1. How to create an LLM and Agent
2. How to use built-in tools (Terminal, FileEditor, TaskTracker)
3. **MCP integration** - Add Tavily search with just config (no code!)
4. **Built-in persistence** - Auto-save & resume with 2 parameters

**Why OpenHands SDK?**
- **MCP built-in** - Connect to any MCP server with config, not code
- **Persistence built-in** - No graph setup, no external DB, just 2 params
- Production-ready with type safety (Pydantic)
- Built-in tools for coding tasks
- Works with any LLM (OpenAI, Anthropic, etc.)

## Step 1: Install

In [1]:
# Note: Tavily MCP requires Node.js (for npx command)
!pip install openhands-sdk python-dotenv -q

## Step 2: Load Environment

Create a `.env` file:
```
LLM_API_KEY=sk-your-openai-key
LLM_MODEL=openai/gpt-5
TAVILY_API_KEY=tvly-your-tavily-key
```

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

print(f"Model: {os.getenv('LLM_MODEL', 'openai/gpt-5')}")

Model: openai/gpt-5


## Step 3: Create the LLM

In [3]:
from openhands.sdk import LLM

llm = LLM(
    model="openai/gpt-5",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

print(f"✓ LLM ready: {llm.model}")

<frozen abc>:106: DeprecationWarning: You should use `Logger` instead. Deprecated since version 1.39.0 and will be removed in a future release.
<frozen abc>:106: DeprecationWarning: You should use `LoggerProvider` instead. Deprecated since version 1.39.0 and will be removed in a future release.
/Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/opentelemetry/_events/__init__.py:201: DeprecationWarning: You should use `ProxyLoggerProvider` instead. Deprecated since version 1.39.0 and will be removed in a future release.
  _PROXY_EVENT_LOGGER_PROVIDER = ProxyEventLoggerProvider()
2026-01-24 14:13:23,288::lmnr.opentelemetry_lib.litellm::WARNING: Laminar LiteLLM callback is deprecated. LiteLLM is already instrumented by Laminar. This callback will not have any effect. You can safely remove the callback from your code. (__init__.py:83)


✓ LLM ready: openai/gpt-5


## Step 4: Configure MCP Server (Tavily)

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/) lets you add tools via config instead of code.

OpenHands SDK has **built-in MCP support** - just pass a config dict to your Agent!

We'll use the [Tavily MCP Server](https://docs.tavily.com/documentation/mcp) which provides:
- `tavily-search` - Web search with citations
- `tavily-extract` - Extract content from URLs

**Requires:** Node.js installed (for `npx`)

In [4]:
# MCP config - that's it! No custom classes needed.
# Compare this to ~40 lines of Action/Observation/Executor code!

mcp_config = {
    "mcpServers": {
        "tavily": {
            "command": "npx",
            "args": ["-y", "tavily-mcp@0.1.3"],
            "env": {"TAVILY_API_KEY": os.getenv("TAVILY_API_KEY")}
        }
    }
}

print("✓ MCP config ready (Tavily search + extract)")
print("  Tools will be: tavily-search, tavily-extract")

✓ MCP config ready (Tavily search + extract)
  Tools will be: tavily-search, tavily-extract


## Step 5: Create the Agent with MCP Tools

Pass `mcp_config` to the Agent - MCP tools are automatically discovered and added!

In [5]:
from openhands.sdk import Agent, Tool
from openhands.tools.terminal import TerminalTool
from openhands.tools.file_editor import FileEditorTool
from openhands.tools.task_tracker import TaskTrackerTool

agent = Agent(
    llm=llm,
    tools=[
        Tool(name=TerminalTool.name),
        Tool(name=FileEditorTool.name),
        Tool(name=TaskTrackerTool.name),
        # Tavily tools come from MCP - no Tool() entry needed!
    ],
    mcp_config=mcp_config,  # MCP tools auto-discovered
)

print("✓ Agent ready")
print(f"  Built-in tools: {[t.name for t in agent.tools]}")
print(f"  MCP tools: tavily-search, tavily-extract (from Tavily MCP server)")

✓ Agent ready
  Built-in tools: ['terminal', 'file_editor', 'task_tracker']
  MCP tools: tavily-search, tavily-extract (from Tavily MCP server)


## Step 6: Create a Conversation with Persistence

OpenHands SDK has **built-in persistence** - just add 2 parameters:
- `persistence_dir`: Where to save (auto-creates directory)
- `conversation_id`: Unique session ID (use same ID to resume)

**Every event is auto-saved.** If interrupted, re-run with same ID to resume from exact point.

In [6]:
from openhands.sdk import Conversation
import uuid

cwd = os.getcwd()

# Create a deterministic UUID from a name (so you can resume with same ID)
SESSION_NAME = "simple-agent-demo"
conversation_id = uuid.uuid5(uuid.NAMESPACE_DNS, SESSION_NAME)

# Built-in persistence: just 2 parameters!
# Compare to LangGraph which requires: StateGraph, nodes, edges, checkpointer class
conversation = Conversation(
    agent=agent,
    workspace=cwd,
    persistence_dir="./.conversations",     # Where to save state
    conversation_id=conversation_id,        # UUID (use same name to resume)
)

print(f"✓ Conversation ready in: {cwd}")
print(f"✓ Persistence: ./.conversations/{conversation_id.hex}/")

[01/24/26 14:13:23] INFO     Created new conversation a093d890-d5eb-5d90-a2c2-dceaf1cf5a49             ]8;id=748835;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py\state.py]8;;\:]8;id=370673;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/conversation/state.py#302\302]8;;\
                             State: {'id': UUID('a093d890-d5eb-5d90-a2c2-dceaf1cf5a49'), 'workspace':              
                             {'working_dir': '/Users/rajivshah/Code/openhands-deep-agent', 'kind':                 
                             'LocalWorkspace'}, 'persistence_dir':                                                 
                             '.conversations/a093d890d5eb5d90a2c2dceaf1cf5a49', 'max_iterations': 500,             
                             'stuck_detection': True, 'execution_status':                                          
                             <ConversationExecutionStatus.IDLE: 'idle'>, 'confirmation_policy':                    
                             {'kind': 'NeverConfirm'}, 'security_analyzer': None,                                  
                             'activated_knowledge_skills': [], 'blocked_actions': {},                              
                             'blocked_messages': {}, 'stats': {'usage_to_metrics': {}},                            
                             'secret_registry': {'secret_sources': {}}}                                            
                             Agent: {'llm': {'model': 'openai/gpt-5', 'api_key':                                   
                             SecretStr('**********'), 'openrouter_site_url':                                       
                             'https://docs.all-hands.dev/', 'openrouter_app_name': 'OpenHands',                    
                             'num_retries': 5, 'retry_multiplier': 8.0, 'retry_min_wait': 8,                       
                             'retry_max_wait': 64, 'timeout': 300, 'max_message_chars': 30000,                     
                             'temperature': 0.0, 'top_p': 1.0, 'max_input_tokens': 272000,                         
                             'max_output_tokens': 128000, 'stream': False, 'drop_params': True,                    
                             'modify_params': True, 'disable_stop_word': False, 'caching_prompt':                  
                             True, 'log_completions': False, 'log_completions_folder':                             
                             'logs/completions', 'native_tool_calling': True, 'reasoning_effort':                  
                             'high', 'enable_encrypted_reasoning': True, 'prompt_cache_retention':                 
                             '24h', 'extended_thinking_budget': 200000, 'usage_id': 'default',                     
                             'litellm_extra_body': {}}, 'tools': [{'name': 'terminal', 'params': {}},              
                             {'name': 'file_editor', 'params': {}}, {'name': 'task_tracker', 'params':             
                             {}}], 'mcp_config': {'mcpServers': {'tavily': {'command': 'npx', 'args':              
                             ['-y', 'tavily-mcp@0.1.3'], 'env': {'TAVILY_API_KEY':                                 
                             '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'}}}}, 'include_default_tools':             
                             ['FinishTool', 'ThinkTool'], 'system_prompt_filename':                                
                             'system_prompt.j2', 'security_policy_filename': 'security_policy.j2',                 
                             'system_prompt_kwargs': {'llm_security_analyzer': True}, 'kind': 'Agent'}             

[01/24/26 14:13:23] INFO     TaskTrackerExecutor initialized with save_dir:                       ]8;id=353967;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/task_tracker/definition.py\definition.py]8;;\:]8;id=751451;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/task_tracker/definition.py#161\161]8;;\
                             .conversations/a093d890d5eb5d90a2c2dceaf1cf5a49                                       

[01/24/26 14:13:23] INFO     FileEditor initialized with cwd:                                          ]8;id=876739;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py\editor.py]8;;\:]8;id=579911;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/file_editor/editor.py#95\95]8;;\
                             /Users/rajivshah/Code/openhands-deep-agent                                            

[01/24/26 14:13:23] INFO     Auto-detected: Using SubprocessTerminal (tmux not available)            ]8;id=347595;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/factory.py\factory.py]8;;\:]8;id=308132;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/factory.py#120\120]8;;\

[01/24/26 14:13:23] INFO     Using shell: /bin/bash                                      ]8;id=701949;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/subprocess_terminal.py\subprocess_terminal.py]8;;\:]8;id=310424;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/terminal/subprocess_terminal.py#106\106]8;;\

[01/24/26 14:13:24] INFO     TerminalExecutor initialized with working_dir:                              ]8;id=783107;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/impl.py\impl.py]8;;\:]8;id=747884;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/tools/terminal/impl.py#59\59]8;;\
                             /Users/rajivshah/Code/openhands-deep-agent, username: None, terminal_type:            
                             TerminalSession                                                                       

[01/24/26 14:13:25] INFO     Created 2 MCP tools: ['tavily-search', 'tavily-extract']                   ]8;id=846164;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/mcp/utils.py\utils.py]8;;\:]8;id=379756;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/mcp/utils.py#82\82]8;;\

[01/24/26 14:13:25] INFO     Loaded 5 tools from spec: ['terminal', 'file_editor', 'task_tracker',      ]8;id=763;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py\base.py]8;;\:]8;id=678789;file:///Users/rajivshah/Code/openhands-deep-agent/.conda/lib/python3.12/site-packages/openhands/sdk/agent/base.py#281\281]8;;\
                             'tavily-search', 'tavily-extract']                                                    

System Prompt ─────────────────────────────────────────────────────────────────────────────────────────────────────

System Prompt:
You are OpenHands agent, a helpful AI assistant that can interact with a computer to solve tasks.

<ROLE>
* Your primary role is to assist users by executing commands, modifying code, and solving technical problems 
effectively. You should be thorough, methodical, and prioritize quality over speed.
* If the user asks a question, like "why is X happening", don't try to fix the problem. Just give an answer to the 
question.
</ROLE>

<MEMORY>
* Use `AGENTS.md` under the repository root as your persistent memory for repository-specific knowledge and 
context.
* Add important insights, patterns, and learnings to this file to improve future task performance.
* This repository skill is automatically loaded for every conversation and helps maintain context across sessions.
* For more information about skills, see: https://docs.openhands.dev/overview/skills
</MEMORY>

<EFFICIENCY>
* Each action you take is somewhat expensive. Wherever possible, combine multiple actions into a single action, 
e.g. combine multiple bash commands into one, using sed and grep to edit/view multiple files at once.
* When exploring the codebase, use efficient tools like find, grep, and git commands with appropriate filters to 
minimize unnecessary operations.
</EFFICIENCY>

<FILE_SYSTEM_GUIDELINES>
* When a user provides a file path, do NOT assume it's relative to the current working directory. First explore the
file system to locate the file before working on it.
* If asked to edit a file, edit the file directly, rather than creating a new file with a different filename.
* For global search-and-replace operations, consider using `sed` instead of opening file editors multiple times.
* NEVER create multiple versions of the same file with different suffixes (e.g., file_test.py, file_fix.py, 
file_simple.py). Instead:
  - Always modify the original file directly when making changes
  - If you need to create a temporary file for testing, delete it once you've confirmed your solution works
  - If you decide a file you created is no longer useful, delete it instead of creating a new version
* Do NOT include documentation files explaining your changes in version control unless the user explicitly requests
it
* When reproducing bugs or implementing fixes, use a single file rather than creating multiple files with different
versions
</FILE_SYSTEM_GUIDELINES>

<CODE_QUALITY>
* Write clean, efficient code with minimal comments. Avoid redundancy in comments: Do not repeat information that 
can be easily inferred from the code itself.
* When implementing solutions, focus on making the minimal changes needed to solve the problem.
* Before implementing any changes, first thoroughly understand the codebase through exploration.
* If you are adding a lot of code to a function or file, consider splitting the function or file into smaller 
pieces when appropriate.
* Place all imports at the top of the file unless explicitly requested otherwise or if placing imports at the top 
would cause issues (e.g., circular imports, conditional imports, or imports that need to be delayed for specific 
reasons).
</CODE_QUALITY>

<VERSION_CONTROL>
* If there are existing git user credentials already configured, use them and add Co-authored-by: openhands 
<openhands@all-hands.dev> to any commits messages you make. if a git config doesn't exist use "openhands" as the 
user.name and "openhands@all-hands.dev" as the user.email by default, unless explicitly instructed otherwise.
* Exercise caution with git operations. Do NOT make potentially dangerous changes (e.g., pushing to main, deleting 
repositories) unless explicitly asked to do so.
* When committing changes, use `git status` to see all modified files, and stage all files necessary for the 
commit. Use `git commit -a` whenever possible.
* Do NOT commit files that typically shouldn't go into version control (e.g.

✓ Conversation ready in: /Users/rajivshah/Code/openhands-deep-agent
✓ Persistence: ./.conversations/a093d890d5eb5d90a2c2dceaf1cf5a49/


## Step 7: Run a Research Query (uses Tavily search!)

In [7]:
# Ask the agent to research something - it will use tavily-search from MCP
conversation.send_message("""
Estimate the main profit drivers for NVIDIA in 2026.
Use web search to find current information, then summarize your findings.
""")
conversation.run()

Message from User ─────────────────────────────────────────────────────────────────────────────────────────────────


Estimate the main profit drivers for NVIDIA in 2026.
Use web search to find current information, then summarize your findings.


Tokens: ↑ input 0 • cache hit N/A • ↓ output 0 • $ 0.00

Agent Action (Not Executed) ───────────────────────────────────────────────────────────────────────────────────────

Thought:
I'll search recent analyses on NVIDIA's 2026 profit drivers.

Reasoning:
Function call:
- tavily-search (call_qg4PLdrpyTEVJQH6hBAs3rc1)


Tokens: ↑ input 6.29K • cache hit 0.00% •  reasoning 960 • ↓ output 1.02K • $ 0.0181

Agent Error ───────────────────────────────────────────────────────────────────────────────────────────────────────

Error Details:
Error validating args {"security_risk":"LOW","summary":"Search news and analysis on NVIDIA 2026 profit drivers 
Blackwell"} for tool 'tavily-search': 1 validation error for MCPTavilySearchAction
query
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

Tokens: ↑ input 6.29K • cache hit 0.00% •  reasoning 960 • ↓ output 1.02K • $ 0.0181

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search for NVIDIA 2026 profit drivers and outlook

Reasoning:
Action: MCPToolAction

Arguments:
  data: {'query': 'NVIDIA 2026 profit drivers Blackwell GB200 data center revenue outlook 2026 NVIDIA guidance 
Blackwell GB200 AI Enterprise networking InfiniBand DGX Cloud DRIVE Thor Omniverse'}
  kind: "MCPToolAction"

Tokens: ↑ input 13.72K • cache hit 52.24% •  reasoning 1.15K • ↓ output 1.29K • $ 0.0220

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: tavily-search
Result:
[MCP Tool 'tavily-search' Observation]
[Tool 'tavily-search' executed.]
Detailed Results:

Title: NVIDIA Q3 FY 2026: Record Data Center Revenue, Higher Q4 Guide
URL: https://futurumgroup.com/insights/nvidia-q3-fy-2026-record-data-center-revenue-higher-q4-guide/
Content: **Analyst Take:** NVIDIA’s Q3 FY 2026 results reflect durable, multi-segment AI infrastructure demand 
anchored by the Blackwell platform, CUDA software stack, and a growing network and systems footprint. Enterprise 
platforms such as ServiceNow, SAP, and Palantir (with CUDA-X and AI models) are integrating NVIDIA’s stack, while 
industry use cases in healthcare and productivity (such as Abridge, OpenEvidence, and Cursor) point to ROI-led 
adoption. With announced AI factory projects approximating 5 million GPUs and networking scaling across NVLink, 
Ethernet, and InfiniBand, NVIDIA’s full-stack approach strengthens visibility across cloud, sovereign, and 
enterprise builds. Content published under the Futurum Research byline incorporates both human and AI-generated 
information, always with editorial oversight and review from the expert Futurum Research team to ensure quality, 
accuracy, and relevance. Keith Kirkpatrick, Research Director at Futurum, examines ServiceNow’s multi-year 
collaboration with OpenAI, highlighting a shift toward agentic AI embedded in core enterprise workflows.... Brendan
Burke, Research Director at Futurum, explores how Tesla’s dual-foundry strategy for its AI5 chip enables record 
production scale and could make multi-foundry production the new standard for AI silicon....

Title: Will NVDA's Blackwell Platform Support Its Data Center Revenue ...
URL: https://finance.yahoo.com/news/nvdas-blackwell-platform-support-data-123500435.html
Content: **NVIDIA Corporation**’s NVDA Blackwell platform is becoming a key driver of its Data Center revenue 
growth. The ongoing ramp-up of the Blackwell architecture is central to NVIDIA’s Data Center business growth. 
NVIDIA also sees the growing adoption of agentic AI, long-context workloads and advanced inference systems, all of 
which depend on high-performance GPU clusters, as key catalysts for long-term growth. The growing demand for the 
company’s AI chips used in data centers is likely to continue aiding its Data Center business performance. Our 
model estimate for fiscal 2026 Data Center revenues is currently pegged at $190 billion, indicating a 
year-over-year increase of 65%. ## NVIDIA’s Competitors in the AI Data Center Space. **Advanced Micro Devices, 
Inc.** AMD and **Intel Corporation** INTC are two major companies that are competing closely with NVIDIA in the AI 
data center space. The Zacks Consensus Estimate for NVIDIA’s fiscal 2026 and 2027 earnings implies a year-over-year
increase of approximately 55.9% and 55.2%, respectively. NVIDIA Corporation (NVDA) : Free Stock Analysis Report.

Title: What To Expect From NVIDIA's Report - Texhoma Wheat Growers, Inc.
URL: https://www.wheatgrowersinc.com/news/story/37149527/earnings-preview-what-to-expect-from-nvidia-s-report
Content: With a market cap of $4.3 trillion, NVIDIA Corporation (NVDA) is a global computing infrastructure company
that provides advanced graphics, compute, and networking solutions across gaming, data center, professional 
visualization, and automotive markets. For fiscal 2026, analysts predict NVIDIA to report EPS of $4.43, an increase
of 51.2% from $2.93 in fiscal 2025. NVIDIA reported record Q3 2026 results on Nov. 19, posting EPS of $1.30 and 
record revenue of $57 billion. The company also posted record Data Center revenue of $51.2 billion, up 25% from Q2 
and 66% from a year ago, driven by surging demand for Blackwell AI chips and cloud GPUs that are sold out. In 
addition, NVIDIA issued a strong Q4 outlook with expected revenue of $65 billion and highlighted major AI 
in

...

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Search NVIDIA Spectrum-X Ethernet run rate and outlook sources

Reasoning:
Action: MCPToolAction

Arguments:
  data: {'query': 'NVIDIA Spectrum-X run rate $10 billion 2025 Nvidia networking revenue outlook 2026 Kress 
comments Spectrum-X Ethernet AI networking platform'}
  kind: "MCPToolAction"

Tokens: ↑ input 468.18K • cache hit 90.69% •  reasoning 2.88K • ↓ output 4.25K • $ 0.1501

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: tavily-search
Result:
[MCP Tool 'tavily-search' Observation]
[Tool 'tavily-search' executed.]
Detailed Results:

Title: NVIDIA's Spectrum-X Adoption Rises: Will It Boost Networking ...
URL: https://www.nasdaq.com/articles/nvidias-spectrum-x-adoption-rises-will-it-boost-networking-revenues
Content: **NVIDIA Corporation** NVDA is seeing rising adoption of its Spectrum-X platform as artificial 
intelligence (AI) data centers grow larger and more complex. Spectrum-X is designed to enhance Ethernet networking 
performance for AI workloads, enabling graphics processing units (GPUs) exchange data faster and more efficiently. 
As more customers build large AI clusters, networking has become a critical part of overall system performance, 
making Spectrum-X an important growth lever for NVIDIA’s networking revenues. The Zacks Consensus Estimate for 
NVIDIA’s networking revenues in fiscal 2026 is pegged at $29.34 billion, indicating 125% growth from fiscal 2025’s 
$12.99 billion. The Zacks Consensus Estimate for NVIDIA’s fiscal 2026 and 2027 earnings implies a year-over-year 
increase of approximately 55.9% and 57%, respectively. ## Zacks' Research Chief Picks Stock Most Likely to "At 
Least Double". NVIDIA Corporation (NVDA) : Free Stock Analysis Report. Zacks is the leading investment research 
firm focusing on stock research, analysis and recommendations.

Title: Live: Will NVIDIA (NVDA) Continue To Surge After Q2 Earnings
URL: https://247wallst.com/investing/2025/08/27/live-will-nvidia-nvda-continue-to-surge-after-q2-earnings/
Content: ... AI. Networking & Spectrum-X Adoption- Networking revenue surged 64% QoQ to $5B, with Spectrum-X 
Ethernet solutions now annualizing at $8B

Title: Can NVIDIA's End-to-End Stack Keep Driving Networking Revenues?
URL: https://finance.yahoo.com/news/nvidias-end-end-stack-keep-130700022.html
Content: Per our model, the company's revenues from the networking business are estimated to grow 57.7% year over 
year to $20.5 billion in fiscal 2026.

Title: Nvidia posts record data center revenue, will provide Spectrum-X for ...
URL: 
https://www.datacenterdynamics.com/en/news/nvidia-posts-record-data-center-revenue-to-provide-spectrum-x-for-openai
s-stargate-project/
Content: Nvidia posted record-breaking data center revenues of $115.2 billion for FY25, a 142 percent increase on 
the revenue generated by the segment during the

Title: 3 Growth Stocks to Invest $1,000 Right Now - AOL.com
URL: https://www.aol.com/finance/3-growth-stocks-invest-1-010600540.html
Content: Sales of its Spectrum-X enhanced Ethernet solutions have reached an annual run rate of over $10 billion. 
... run generative AI workloads on.

Agent Action ──────────────────────────────────────────────────────────────────────────────────────────────────────

Predicted Security Risk: LOW

Summary: Extract Nasdaq article on Spectrum-X adoption and networking revenue estimates for FY2026

Reasoning:
Action: MCPToolAction

Arguments:
  data: {'urls': 
['https://www.nasdaq.com/articles/nvidias-spectrum-x-adoption-rises-will-it-boost-networking-revenues']}
  kind: "MCPToolAction"

Tokens: ↑ input 514.78K • cache hit 91.38% •  reasoning 2.88K • ↓ output 4.32K • $ 0.1575

Observation ───────────────────────────────────────────────────────────────────────────────────────────────────────

Tool: tavily-extract
Result:
[MCP Tool 'tavily-extract' Observation]
[Tool 'tavily-extract' executed.]
Detailed Results:

Title: NVIDIA's Spectrum-X Adoption Rises: Will It Boost Networking Revenues? | Nasdaq
URL: https://www.nasdaq.com/articles/nvidias-spectrum-x-adoption-rises-will-it-boost-networking-revenues
Content: undefined
Raw Content: Data is currently not available

### We couldn’t find any results matching your search.

Please try using other words for your search or explore other sections of the website for relevant information.

### We’re sorry, we are currently experiencing some issues, please try again later.

Our team is working diligently to resolve the issue. Thank you for your patience and understanding.

[Show more results ->](/search)

[AAPL](/market-activity/stocks/aapl)

[TSLA](/market-activity/stocks/tsla)

[AMZN](/market-activity/stocks/amzn)

[META](/market-activity/stocks/meta)

[AMD](/market-activity/stocks/amd)

[NVDA](/market-activity/stocks/nvda)

[PEP](/market-activity/stocks/pep)

[COST](/market-activity/stocks/cost)

[ADBE](/market-activity/stocks/adbe)

[GOOG](/market-activity/stocks/goog)

[AMGN](/market-activity/stocks/amgn)

[HON](/market-activity/stocks/hon)

[INTC](/market-activity/stocks/intc)

[INTU](/market-activity/stocks/intu)

[NFLX](/market-activity/stocks/nflx)

[ADP](/market-activity/stocks/adp)

[SBUX](/market-activity/stocks/sbux)

[MRNA](/market-activity/stocks/mrna)

[AAPL](/market-activity/stocks/aapl)

[TSLA](/market-activity/stocks/tsla)

[AMZN](/market-activity/stocks/amzn)

[META](/market-activity/stocks/meta)

[AMD](/market-activity/stocks/amd)

[NVDA](/market-activity/stocks/nvda)

[PEP](/market-activity/stocks/pep)

[COST](/market-activity/stocks/cost)

[ADBE](/market-activity/stocks/adbe)

[GOOG](/market-activity/stocks/goog)

[AMGN](/market-activity/stocks/amgn)

[HON](/market-activity/stocks/hon)

[INTC](/market-activity/stocks/intc)

[INTU](/market-activity/stocks/intu)

[NFLX](/market-activity/stocks/nflx)

[ADP](/market-activity/stocks/adp)

[SBUX](/market-activity/stocks/sbux)

[MRNA](/market-activity/stocks/mrna)

[AAPL](/market-activity/stocks/aapl)

[TSLA](/market-activity/stocks/tsla)

[AMZN](/market-activity/stocks/amzn)

[META](/market-activity/stocks/meta)

[AMD](/market-activity/stocks/amd)

[NVDA](/market-activity/stocks/nvda)

[PEP](/market-activity/stocks/pep)

[COST](/market-activity/stocks/cost)

[ADBE](/market-activity/stocks/adbe)

[GOOG](/market-activity/stocks/goog)

[AMGN](/market-activity/stocks/amgn)

[HON](/market-activity/stocks/hon)

[INTC](/market-activity/stocks/intc)

[INTU](/market-activity/stocks/intu)

[NFLX](/market-activity/stocks/nflx)

[ADP](/market-activity/stocks/adp)

[SBUX](/market-activity/stocks/sbux)

[MRNA](/market-activity/stocks/mrna)

[Stocks](/topic/stocks)

# NVIDIA's Spectrum-X Adoption Rises: Will It Boost Networking Revenues?

January 21, 2026 — 07:56 am EST

Written by Anirudha Bhagat for  [Zacks->](/publishers/zacks)

**NVIDIA Corporation** [NVDA](/market-activity/stocks/nvda) is seeing rising adoption of its Spectrum-X platform as
artificial intelligence (AI) data centers grow larger and more complex. Spectrum-X is designed to enhance Ethernet 
networking performance for AI workloads, enabling graphics processing units (GPUs) exchange data faster and more 
efficiently. As more customers build large AI clusters, networking has become a critical part of overall system 
performance, making Spectrum-X an important growth lever for NVIDIA’s networking revenues.

In the third quarter of fiscal 2026, networking revenues surged 162% year over year to $8.19 billion, driven by 
strong demand for AI data center connectivity. As workloads become larger and more complex, the need for faster, 
more efficient interconnects is growing, and Spectrum-X is designed to meet that demand.

Cloud service providers an

Message from Agent ────────────────────────────────────────────────────────────────────────────────────────────────

I’m going to summarize recent reporting and analyst commentary now.

Bottom line
- Primary: Data center AI systems (Blackwell GPUs and full‑stack racks) and fast‑growing AI networking
- Secondary: Gaming/AI PCs and automotive/robotics
- Margin support: Software and services attached to the platform

What recent sources show
1) Data center AI compute remains the core profit engine
- NVIDIA’s Q1 FY2026 revenue was $44.1B, with Data Center at $39.1B; the company guided non‑GAAP gross margin to 
~72% and is “working toward” mid‑70% margins later in the year (press release). Blackwell NVL72 production and 
demand were highlighted as key drivers [NVIDIA IR Q1 FY26 press release, May 28, 2025: investor.nvidia.com; 
nvidianews PDF].
- In Q3 FY2026, Fortune reported data center accounted for about 90% of revenue, and CFO Colette Kress said NVIDIA 
has “visibility to a half a trillion dollars in Blackwell and Rubin revenue from the start of this year through the
end of calendar year 2026,” underscoring that 2026 profits are led by data center chips/systems [Fortune, Nov 19, 
2025].

2) AI networking is a rapidly growing second profit driver
- NVIDIA’s data center networking (NVLink Switch, Spectrum‑X Ethernet, Quantum InfiniBand) surged in 2025–2026 as 
clusters scale. Next Platform’s Q2 FY2026 analysis cites datacenter networking at $7.25B for the quarter and notes 
Spectrum‑X’s annualized run‑rate above $10B, with networking becoming a larger share of platform cost [The Next 
Platform, Aug 27, 2025].
- In Q3 FY2026, a Zacks/Nasdaq piece cited networking revenue up 162% YoY to $8.19B and a FY2026 networking 
estimate of ~$29B (consensus) as Spectrum‑X adoption accelerates among hyperscalers and enterprises [Nasdaq/Zacks, 
Jan 21, 2026].
- NVIDIA said Spectrum‑X and NVLink Switch are a “major new growth vector,” and it is supplying Spectrum‑X to 
flagship AI factory builds (e.g., OpenAI’s “Stargate”) [DataCenterDynamics, Feb 2026].

3) Full‑stack platforms (GB200 NVL72, DGX/HGX) push system ASPs and margins
- NVIDIA’s GB200 Grace Blackwell NVL72 rack‑scale system and NVLink Switch drive higher‑value, integrated sales 
versus standalone GPUs [NVIDIA GB200 NVL72 product page].
- The IR release also flagged NVLink Fusion and Blackwell Ultra as part of the roadmap, which sustains platform 
pricing power and differentiation into 2026 [NVIDIA IR Q1 FY26 press release, May 28, 2025].

4) Software and services support margins, but remain smaller vs. hardware in 2026
- CUDA/CUDA‑X, AI Enterprise/NIM microservices, DGX Cloud Lepton, Omniverse/Cosmos, and enterprise reference stacks
are increasingly bundled and referenced, but NVIDIA doesn’t break out large standalone software revenue; these 
offerings primarily enhance platform stickiness and margins attached to hardware in 2026 [NVIDIA IR Q1 FY26 press 
release].

5) Secondary contributors: Gaming/AI PC and Automotive/Robotics
- Gaming/AI PC: Q1 FY2026 gaming revenue reached a record $3.8B, aided by RTX 50‑series launches; still small 
relative to data center but profitable and steady [NVIDIA IR Q1 FY26 press release].
- Automotive/Robotics: Q1 FY2026 automotive was $567M (+72% YoY per CNBC), with DRIVE/Thor and robotics platforms 
ramping; meaningful growth potential, but a modest 2026 profit contributor vs. data center [CNBC, May 28, 2025; 
NVIDIA IR Q1 FY26 press release].

What this implies for 2026 profit drivers
- #1 Driver: Data center AI compute platforms (Blackwell B200/B100, GB200, NVL72) and DGX/HGX systems. Expect this 
to represent the overwhelming majority of 2026 profits given mix, supply, and demand visibility (Fortune cites ~90%
DC mix in Q3 FY2026) [Fortune; NVIDIA IR].
- #2 Driver: Data center networking (NVLink Switch, Spectrum‑X Ethernet, Quantum InfiniBand). A rapidly growing, 
high‑attach business that expands the total platform value; quarterly networking r

## OpenHands SDK Advantages

### 1. MCP Integration (No Code!)

We added Tavily search with just a config dict - no custom classes:

```python
# OpenHands SDK: 10 lines of config
mcp_config = {
    "mcpServers": {
        "tavily": {"command": "npx", "args": ["-y", "tavily-mcp"], ...}
    }
}
agent = Agent(llm=llm, mcp_config=mcp_config)
```

vs. Custom tool approach (~40 lines): See `01_deep_research.ipynb` for the Action/Observation/Executor pattern when you need full control.

### 2. Built-in Persistence

Your conversation is saved to `.conversations/<uuid>/`. Resume anytime:

```python
import uuid
# Same name = same UUID = auto-restore
conversation_id = uuid.uuid5(uuid.NAMESPACE_DNS, "simple-agent-demo")
conversation = Conversation(
    agent=agent,
    persistence_dir="./.conversations",
    conversation_id=conversation_id,
)
conversation.run()  # Continues where it left off
```

### Compare to LangGraph

| Feature | OpenHands SDK | LangGraph |
|---------|---------------|-----------|
| **Add tools** | Config dict (MCP) | Code or MCP |
| **Persistence** | 2 params | Graph + checkpointer class |
| **Graph required?** | ❌ No | ✅ Yes |
| **Lines for basic agent** | ~15 | ~40-50 |

**OpenHands SDK: MCP + Persistence without the graph overhead.**

---

## Next Steps

- `01_deep_research.ipynb` - Multi-agent workflow with custom tools
- `02_parallel_research.ipynb` - Parallel sub-agent delegation